# PyBADS Example 5: Extended usage

In this example, we will show PyBADS at work on a multimodal target and showcase some additional features.

This notebook is Part 5 of a series of notebooks in which we present various example usages for BADS with the PyBADS package.

In [1]:
import numpy as np
from pybads.bads.bads import BADS

## 1. Problem setup

In this example, we are going to optimize the *six-hump camelback function*, which has six local minima, two of which are global minima.

Note that, in most realistic scenarios, you would not know whether your problem has only a single local minimum (which is also the global minimum). In practice, many optimization problems exhibit *multiple* local minima.

In [2]:
def camelback6(x):
    """Six-hump camelback function."""
    x_2d = np.atleast_2d(x)
    x1 = x_2d[:,0]
    x2 = x_2d[:,1]
    f = (4 - 2.1*(x1*x1) + (x1*x1*x1*x1)/3.0)*(x1*x1) + x1*x2 + (-4 + 4*(x2*x2))*(x2*x2)
    return f

lb = np.array([-3, -2])       # Lower bounds
ub = np.array([3, 2])         # Upper bounds
plb = np.array([-2.9, -1.9])  # Plausible lower bounds
pub = np.array([2.9, 1.9])    # Plausible upper bounds

options = {
    "display" : 'off',             # We switch off the printing
    "uncertainty_handling": False, # Good to specify that this is a deterministic function
}

## 2. Run the optimization

PyBADS is not a global optimization algorithm in that there is no guarantee that a single run would return the global optimum (in fact, this is true of almost all algorithms). The gold rule of optimization, regardless of optimization algorithm, is to always rerun the optimization multiple times from different starting points (a multi-start strategy), to explore the landscape of the target and gain some confidence about the results.

Below, we rerun PyBADS `num_opts` times from different starting points and store the results of each run, which we will examine later. Note that we switched off PyBADS default printing.

Also note that each optimization uses a different `BADS` object (the general rule is: one BADS instance per optimization).

In [3]:
num_opts = 10
optimize_results = []
x_vec = np.zeros((num_opts,lb.shape[0]))
fval_vec = np.zeros(num_opts)

for opt_count in range(num_opts):
    print('Running optimization ' + str(opt_count) + '...')
    x0 = np.random.uniform(low=plb, high=pub)
    bads = BADS(camelback6, x0, lb, ub, plb, pub, options=options)
    optimize_results.append(bads.optimize())
    x_vec[opt_count] = optimize_results[opt_count].x
    fval_vec[opt_count] = optimize_results[opt_count].fval

Running optimization 0...
Function value at minimum: -1.0316188687309946

Running optimization 1...
Function value at minimum: -1.031627209412116

Running optimization 2...
Function value at minimum: -1.031624709878634

Running optimization 3...
Function value at minimum: -1.0316266405918255

Running optimization 4...
Function value at minimum: -1.031628154388721

Running optimization 5...
Function value at minimum: -1.031627526801249

Running optimization 6...
Function value at minimum: -1.0316222000749877

Running optimization 7...
Function value at minimum: -1.0316274584136575

Running optimization 8...
Function value at minimum: -1.0316266950914683

Running optimization 9...
Function value at minimum: -1.031598130313844



## 3. Results and conclusions

First, we inspect the results. In this example, the target function has two equally-good solutions, 
$$
x^\star = \left\{ (0.0898, -0.7126), (-0.0898, 0.7126) \right\}, \qquad f(x^\star) = -1.0316
$$
which should be represented in the set of results. 

Importantly, we should find below that (almost) all solutions are very close in function value, suggesting that we found the minimizers of the target.

In [4]:
print('Found solutions:')
print(x_vec)

print('Function values at solutions:')
print(fval_vec)

Found solutions:
[[-0.09141472  0.71270269]
 [ 0.09040388 -0.7127438 ]
 [-0.08887769  0.712464  ]
 [ 0.09051437 -0.71260872]
 [-0.08956575  0.71261778]
 [-0.08951874  0.71238403]
 [ 0.0903595  -0.71348572]
 [ 0.09020462 -0.71243477]
 [-0.09045906  0.71250725]
 [ 0.08708241 -0.71281016]]
Function values at solutions:
[-1.03161887 -1.03162721 -1.03162471 -1.03162664 -1.03162815 -1.03162753
 -1.0316222  -1.03162746 -1.0316267  -1.03159813]


We now take the best result of the optimization:

In [5]:
idx_best = np.argmin(fval_vec)
result_best = optimize_results[idx_best]

x_min = result_best['x']
fval = result_best['fval']

print(f"BADS minimum at x_min = {x_min.flatten()}")
print(f"Function value at minimum fval = {fval}")

BADS minimum at x_min = [-0.08956575  0.71261778]
Function value at minimum fval = -1.031628154388721


The best result indeed matches $f^\star = -1.0316$.

The `OptimizeResult` object returned by PyBADS contains further information about the run:

In [6]:
result_best

{'fun': <function __main__.camelback6(x)>,
 'non_box_cons': None,
 'target_type': 'deterministic',
 'problem_type': 'bound constraints',
 'iterations': 8,
 'func_count': 85,
 'mesh_size': 0.0009765625,
 'overhead': 18951355.422884997,
 'algorithm': 'Bayesian adaptive direct search',
 'yval_vec': None,
 'ysd_vec': None,
 'x0': array([[ 0.68893913, -1.51155983]]),
 'x': array([-0.08956575,  0.71261778]),
 'fval': -1.031628154388721,
 'fsd': 0,
 'total_time': 1.591913939522339,
 'success': True,
 'random_seed': None,
 'version': '0.0.1',
 'message': 'Optimization terminated: change in the function value less than options.TolFun.'}

## Example 5: Full code

See [here](./src/pybads_example_5_extended_usage.py) for a Python file with the code used in this example, with no extra fluff.